In [2]:
import sympy

# Quasistatic Poroelasticity

The governing equations are

\begin{gather}
% Solution
\vec{s}^{T} = \left(\vec{u} \quad p \quad \vec{\epsilon}_v \right) \\
% Displacement
\vec{f}(\vec{x},t) + \nabla \cdot \boldsymbol{\sigma}(\vec{u},p) = 0 \text{ in } \Omega \\
% Pressure
\frac{\partial \zeta(\vec{u},p)}{\partial t } - \gamma(\vec{x},t) + \nabla \cdot \vec{q}(p) = 0 \text{ in } \Omega \\
% Neumann traction
\boldsymbol{\sigma} \cdot \vec{n} = \vec{\tau}(\vec{x},t) \text{ on } \Gamma_{\tau} \\
% Dirichlet displacement
\vec{u} = \vec{u}_{0}(\vec{x}, t) \text{ on } \Gamma_{u} \\
% Neumann flow
\vec{q} \cdot \vec{n} = q_{0}(\vec{x}, t) \text{ on } \Gamma_{q} \\
% Dirichlet pressure
p = p_{0}(\vec{x},t) \text{ on } \Gamma_{p}
\end{gather}

where

\begin{gather}
%
  \vec{q}(p) = -\frac{\boldsymbol{k}}{\mu_{f}}(\nabla p - \vec{f}_f), \\
%
  \zeta(\vec{u},p) = \alpha (\nabla \cdot \vec{u}) + \frac{p}{M}, \\
%
  \boldsymbol{\sigma}(\vec{u},p) = \boldsymbol{C}:\boldsymbol{\epsilon} - \alpha p \boldsymbol{I}
  = \lambda \boldsymbol{I} \epsilon_{v} + 2 \mu \boldsymbol{\epsilon}  - \alpha \boldsymbol{I} p, \\
\lambda = K_{d} - \frac{2}{3} \mu, \\
  \frac{1}{M} = \frac{\alpha-\phi}{K_s} + \frac{\phi}{K_f}, \\
  \alpha = 1 - \frac{K_d}{K_f}, \\
\epsilon_{v} = \nabla \cdot \vec{u},
\end{gather}

$M$ denotes the Biot modulus, $\alpha$ denotes the Biot coefficient, $1/M$ is the specific storage coefficient at constant strain, $K_d$ denotes the bulk modulus of the drained system, $K_s$ denotes the bulk modulus of the solid, and $K_f$ denotes the bulk modulus of the fluid, $\mu$ denotes the shear modulus, and $\epsilon_{v}$ denotes the volumetric strain.

## Test Case: Gravitational Body Forces

We consider the steady-state solution for loading from gravitational body forces,

\begin{gather}
q_x = q_y = 0 \Rightarrow \nabla p + \rho_f g = 0 \\
p = -\rho_f g y
\end{gather}

Solving the elasticity equation leads to

\begin{aligned}
\sigma_{yy} &= \rho_b g y, \\
\sigma_{xx} &= \sigma_{zz} = \frac{\lambda}{\lambda+2\mu} \rho_b g y + \frac{2\mu}{\lambda+2\mu} \alpha \rho_f g y, \\
\sigma_{xy} &= 0, \\
\epsilon_v &= -\frac{1}{\lambda + 2\mu} \left( \rho_b - \alpha \rho_f \right) g y, \\
u_x &= 0, \\
u_y(y) &= -\frac{1}{2} \frac{1}{\lambda+2\mu} \left( \rho_b - \alpha \rho_f \right) g (l^2-y^2)\\
\end{aligned}

### Verify the analytical solution

In [113]:
# Define symbolic variable
x, y = sympy.symbols("x, y")

# Define symbolic constants
g, l = sympy.symbols("g, l")

# Material parameters
λ, μ, ϕ, α, μf, k, ρb, ρf, M = sympy.symbols("λ, μ, ϕ, α, μf, k, ρb, ρf, M")

# Analytical solution
ux = 0*x
uy = -0.5/(λ+2*μ) * (ρb-α*ρf) * g * (l**2-y**2)
p = -ρf * g * y

# Derivatives for governing equations
ux_x = ux.diff(x)
ux_y = ux.diff(y)
uy_x = uy.diff(x)
uy_y = uy.diff(y)
p_x = p.diff(x)
p_y = p.diff(y)
grad_p = sympy.Matrix([p_x, p_y])

# Body force for fluid phase
ff = sympy.Matrix([0., -ρf * g])

# Body force for solid phase
fs = sympy.Matrix([0., -ρb * g])

# Darcy flux; Generalized Dacy's law
q = -(k/μf)*( grad_p - ff)

# Strain
ϵxy = (ux_y + uy_x) / 2
ϵ = sympy.Matrix([[ux_x, ϵxy],[ϵxy, uy_y]])
ϵv = sympy.trace(ϵ)

# Stress
σ  = λ * ϵv * sympy.eye(2) + 2 * μ * ϵ - α * sympy.eye(2) * p

# Variation of fluid content
ζ = α * ϵv + p/M

# Divergence of stress
div_σ = sympy.Matrix([σ[0,0].diff(x) + σ[0,1].diff(y), σ[1,0].diff(x) + σ[1,1].diff(y)])

# Divergence of flux
div_q = q[0].diff(x) + q[1].diff(y)

In [114]:
sympy.simplify(σ)

Matrix([
[g*y*(2.0*α*μ*ρf + 1.0*λ*ρb)/(1.0*λ + 2.0*μ),          0],
[                                          0, 1.0*g*y*ρb]])

In [ ]:
ϵv

1.0*g*y*(-α*ρf + ρb)/(λ + 2*μ)

In [115]:
sympy.simplify(q)

Matrix([
[0],
[0]])

In [116]:
sympy.simplify(div_σ + fs)

Matrix([
[0],
[0]])

In [118]:
grad_p + -ff

Matrix([
[0],
[0]])

## Test Case: Body Forces

We consider the steady-state solution for loading from body forces,

\begin{gather}
q_x = q_y = 0 \Rightarrow \nabla p + \vec{f} = 0 \\
p = -\vec{f}
\end{gather}

Solving the elasticity equation leads to

\begin{aligned}
\sigma_{xx} &= f_x x, \\
\sigma_{yy} &= \sigma_{zz} = \frac{\lambda}{\lambda+2\mu} f_x x + \frac{2\mu}{\lambda+2\mu} \alpha f_x x, \\
\sigma_{xy} &= 0, \\
\epsilon_v &= -\frac{1}{\lambda + 2\mu} \left( 1.0 - \alpha \right) f_x x, \\
u_x(x) &= -\frac{1}{2} \frac{1}{\lambda+2\mu} \left( 1.0 - \alpha \right) f_x (l^2-x^2), \\
u_y &= 0. \\
\end{aligned}

In [159]:
# Define symbolic variable
x, y = sympy.symbols("x, y")

# Define symbolic constants
l, fx = sympy.symbols("l, fx")

# Material parameters
λ, μ, ϕ, α, μf, k, ρb, ρf, M = sympy.symbols("λ, μ, ϕ, α, μf, k, ρb, ρf, M")

# Analytical solution
ux = 0.5/(λ+2*μ) * (1.0-α) * fx * (l**2-x**2)
uy = 0*x
p = fx * x

# Derivatives for governing equations
ux_x = ux.diff(x)
ux_y = ux.diff(y)
uy_x = uy.diff(x)
uy_y = uy.diff(y)
p_x = p.diff(x)
p_y = p.diff(y)
grad_p = sympy.Matrix([p_x, p_y])

# Body force for fluid phase
ff = sympy.Matrix([fx, 0.])

# Body force for solid phase
fs = sympy.Matrix([fx, 0.])

# Darcy flux; Generalized Dacy's law
q = -(k/μf)*( grad_p - ff)

# Strain
ϵxy = (ux_y + uy_x) / 2
ϵ = sympy.Matrix([[ux_x, ϵxy],[ϵxy, uy_y]])
ϵv = sympy.trace(ϵ)

# Stress
σ  = λ * ϵv * sympy.eye(2) + 2 * μ * ϵ - α * sympy.eye(2) * p

# Variation of fluid content
ζ = α * ϵv + p/M

# Divergence of stress
div_σ = sympy.Matrix([σ[0,0].diff(x) + σ[0,1].diff(y), σ[1,0].diff(x) + σ[1,1].diff(y)])

# Divergence of flux
div_q = q[0].diff(x) + q[1].diff(y)

In [160]:
sympy.simplify(σ)

Matrix([
[-1.0*fx*x,                                       0],
[        0, fx*x*(-2.0*α*μ - 1.0*λ)/(1.0*λ + 2.0*μ)]])

In [161]:
ϵv

-1.0*fx*x*(1.0 - α)/(λ + 2*μ)

In [162]:
sympy.simplify(q)

Matrix([
[0],
[0]])

In [163]:
sympy.simplify(div_σ + fs)

Matrix([
[0],
[0]])

In [164]:
grad_p - ff

Matrix([
[0],
[0]])

## Test Case: Gravitation and Body Forces

We consider the steady-state solution for loading from gravity and a uniform, constant body force in the +y direction,

\begin{gather}
q_x = q_y = 0 \Rightarrow \nabla p + \rho_f g - f_y = 0 \\
p = (f_y -\rho_f g) y
\end{gather}

Solving the elasticity equation leads to

\begin{aligned}
\sigma_{yy} &= (\rho_b g - f_y) y, \\
\sigma_{xx} &= \sigma_{zz} = \frac{\lambda}{\lambda+2\mu} (\rho_b g - f_y) y + \frac{2\mu}{\lambda+2\mu} \alpha (\rho_f g - f_y) y, \\
\sigma_{xy} &= 0, \\
\epsilon_v &= -\frac{1}{\lambda + 2\mu} \left( (\rho_b - \alpha \rho_f) g - (1 - \alpha) f_y \right) y, \\
u_x &= 0, \\
u_y(y) &= -\frac{1}{2} \frac{1}{\lambda+2\mu} \left( (\rho_b - \alpha \rho_f) g - (1 - \alpha) f_y  \right) (l^2-y^2)\\
\end{aligned}

In [4]:
# Define symbolic variable
x, y = sympy.symbols("x, y")

# Define symbolic constants
g, l, fy = sympy.symbols("g, l, fy")

# Material parameters
λ, μ, ϕ, α, μf, k, ρb, ρf, M = sympy.symbols("λ, μ, ϕ, α, μf, k, ρb, ρf, M")

# Analytical solution
ux = 0*x
uy = -0.5/(λ+2*μ) * ((ρb-α*ρf) * g - (1-α) * fy ) * (l**2-y**2)
p = -(ρf * g - fy) * y

# Derivatives for governing equations
ux_x = ux.diff(x)
ux_y = ux.diff(y)
uy_x = uy.diff(x)
uy_y = uy.diff(y)
p_x = p.diff(x)
p_y = p.diff(y)
grad_p = sympy.Matrix([p_x, p_y])

# Body force for fluid phase
ff = sympy.Matrix([0., -ρf * g + fy])

# Body force for solid phase
fs = sympy.Matrix([0., -ρb * g + fy])

# Darcy flux; Generalized Dacy's law
q = -(k/μf)*( grad_p - ff)

# Strain
ϵxy = (ux_y + uy_x) / 2
ϵ = sympy.Matrix([[ux_x, ϵxy],[ϵxy, uy_y]])
ϵv = sympy.trace(ϵ)

# Stress
σ  = λ * ϵv * sympy.eye(2) + 2 * μ * ϵ - α * sympy.eye(2) * p

# Variation of fluid content
ζ = α * ϵv + p/M

# Divergence of stress
div_σ = sympy.Matrix([σ[0,0].diff(x) + σ[0,1].diff(y), σ[1,0].diff(x) + σ[1,1].diff(y)])

# Divergence of flux
div_q = q[0].diff(x) + q[1].diff(y)

In [5]:
sympy.simplify(σ)

Matrix([
[y*(-α*(fy - g*ρf)*(λ + 2*μ) + 1.0*λ*(fy*(α - 1) - g*(α*ρf - ρb)))/(λ + 2*μ),                  0],
[                                                                          0, 1.0*y*(-fy + g*ρb)]])

In [7]:
sympy.simplify(ϵv)

1.0*y*(fy*(α - 1) - g*(α*ρf - ρb))/(λ + 2*μ)

In [8]:
sympy.simplify(q)

Matrix([
[0],
[0]])

In [9]:
sympy.simplify(div_σ + fs)

Matrix([
[0],
[0]])

In [10]:
grad_p - ff

Matrix([
[0],
[0]])